In [1]:
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.applications.resnet import ResNet50, preprocess_input,ResNet101
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.applications.resnet import ResNet50, ResNet101
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
import os
import numpy as np
import tensorflow
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix,classification_report

2023-11-05 21:23:03.780731: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 21:23:03.807770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input


# CT scan dataset paths
ct_train_path = "/u/45/muhammu2/data/Desktop/train/"
ct_validation_path = "/u/45/muhammu2/data/Desktop/validation/"
ct_test_path = "/u/45/muhammu2/data/Desktop/test/"

# Number of classes depends on the dataset (2 for the CT scan)
ct_num_classes = 2

# Image size depends on the pretrained model (224 for InceptionV3)
image_size = 224
batch_size = 16
epoch_num = 15

# Define a data generator for CT scan training
ct_train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)

ct_train_generator = ct_train_datagen.flow_from_directory(
    ct_train_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Define a data generator for CT scan validation
ct_validation_generator = ct_train_datagen.flow_from_directory(
    ct_validation_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Define a data generator for CT scan testing
ct_test_datagen = image.ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_input)

ct_test_generator = ct_test_datagen.flow_from_directory(
    ct_test_path,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')


Found 720 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1800 images belonging to 2 classes.


In [3]:
	#Create model
	model = Sequential()
	#model.add(InceptionV3(include_top=False, pooling='max', weights='imagenet')) 	# entire resnet model is the first layer!
	#model.add(ResNet50(include_top=False, pooling='max', weights='imagenet')) 	
 	#model.add(ResNet101(include_top=False, pooling='max', weights='imagenet')) 
	model.add(MobileNetV2(include_top=False, pooling='max', weights='imagenet')) 	
 	#model.add(DenseNet121(include_top=False, pooling='max', weights='imagenet'))
	model.add(Dense(ct_num_classes, activation='softmax'))
	model.layers[0].trainable = False		# entire resnet layer is not trainable; only the final dense layer is trained
	opt= keras.optimizers.Adam(learning_rate=0.1)
	model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
	model.summary()

2023-11-05 21:23:11.940342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 21:23:12.052127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-05 21:23:12.052347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 1280)              2257984   
 tional)                                                         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [4]:
steps_per_epoch = ct_train_generator.samples // batch_size
validation_steps = ct_validation_generator.samples // batch_size

In [5]:
model.fit(
    ct_train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epoch_num,  # Use the previously defined epoch_num
    validation_data=ct_validation_generator,
    validation_steps=validation_steps,
    verbose=1
)


Epoch 1/15


2023-11-05 21:23:19.890901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2023-11-05 21:23:20.306774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-05 21:23:20.397403: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555e28be69e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-05 21:23:20.397423: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A2000 8GB Laptop GPU, Compute Capability 8.6
2023-11-05 21:23:20.424017: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 4/45 [=>............................] - ETA: 1s - loss: 93.5014 - accuracy: 0.6562 

2023-11-05 21:23:20.631221: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 5s 67ms/step - loss: 62.9027 - accuracy: 0.6542 - val_loss: 16.9009 - val_accuracy: 0.7913
Epoch 2/15
45/45 [==============================] - 3s 57ms/step - loss: 28.6680 - accuracy: 0.6778 - val_loss: 22.2390 - val_accuracy: 0.7893
Epoch 3/15
45/45 [==============================] - 3s 57ms/step - loss: 15.2043 - accuracy: 0.6750 - val_loss: 8.0161 - val_accuracy: 0.7893
Epoch 4/15
45/45 [==============================] - 3s 57ms/step - loss: 20.2183 - accuracy: 0.6625 - val_loss: 21.3670 - val_accuracy: 0.7893
Epoch 5/15
45/45 [==============================] - 3s 57ms/step - loss: 7.5600 - accuracy: 0.7236 - val_loss: 6.4985 - val_accuracy: 0.7903
Epoch 6/15
45/45 [==============================] - 3s 57ms/step - loss: 19.9048 - accuracy: 0.6639 - val_loss: 35.8600 - val_accuracy: 0.7903
Epoch 7/15
45/45 [==============================] - 3s 57ms/step - loss: 12.8141 - accuracy: 0.6972 - val_loss: 18.5965 - val_accuracy: 0.2631
Epoch 8/15
45

In [7]:
from sklearn.metrics import roc_curve

true_labels_validation = ct_validation_generator.classes  # True labels (0 or 1)
predicted_probabilities_validation = model.predict(ct_validation_generator)  # Predicted probabilities for the positive class

# Compute the ROC curve on the validation set
fpr, tpr, thresholds = roc_curve(true_labels_validation, predicted_probabilities_validation[:, 1])
eer_threshold = thresholds[np.argmin(np.abs(1 - tpr - fpr))]
eer = fpr[np.argmin(np.abs(1 - tpr - fpr))] * 100  # EER as a percentage

print(f"Equal Error Rate (EER) on validation set: {eer:.2f}%")

63/63 [==============================] - 2s 22ms/step
Equal Error Rate (EER) on validation set: 48.61%


In [8]:
predicted_probabilities = model.predict(ct_test_generator)[:, 1]  # Predicted probabilities for the positive class (spoofed)
true_labels = ct_test_generator.classes  # True labels (0 for genuine, 1 for spoofed)
threshold = eer_threshold  # You can adjust this threshold as needed
binary_predictions = [1 if prob >= threshold else 0 for prob in predicted_probabilities]
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(true_labels, binary_predictions)
precision = precision_score(true_labels, binary_predictions)
recall = recall_score(true_labels, binary_predictions)
f1 = f1_score(true_labels, binary_predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")



113/113 [==============================] - 9s 80ms/step
Accuracy: 0.26
Precision: 0.20
Recall: 0.90
F1 Score: 0.33


In [9]:
# Evaluation for testing set for HTER
predicted_probabilities = model.predict(ct_test_generator)[:, 1]  # Predicted probabilities for the positive class (spoofed)
numericLabels = np.array([1 if label == 0 else 0 for label in true_labels])

real_scores1 = predicted_probabilities[numericLabels == 1]
attack_scores2 = predicted_probabilities[numericLabels == 0]

threshold = eer_threshold  # Replace with your chosen threshold

FAR = np.sum(attack_scores2 > threshold) / len(attack_scores2) * 100
FRR = np.sum(real_scores1 <= threshold) / len(real_scores1) * 100
HTER4 = (FAR + FRR) / 2

# Display HTER
print(f"Half Total Error Rate (HTER) for Face Anti-Spoofing: {HTER4:.2f}%")


113/113 [==============================] - 8s 75ms/step
Half Total Error Rate (HTER) for Face Anti-Spoofing: 51.01%
